# Processing Site Lidar

In this notebook we preprocess out lidar sites. This includes:
- Reprojection
- Cropping to the site geometry
- (Optional) classifying ground points
- Computing height above ground
- Remove statistical outliers (noise)
- Saving cloud as a cloud optimised point cloud (COPC)

To process the data we will use PDAL pipelines.
We will also use dask to run the processing in parallel.

In [1]:
from pathlib import Path
import json

import geopandas as gpd
import pdal
import pandas as pd

## PDAL Pipeline Template

Each run of PDAL processing is defined by a pipeline represented by a JSON string. Our processing for each site is mostly the same. Only a few site varaibles will change each run (e.g. name of input file, polygon to clip to). To replace variables we'll use the Jinja template engine. Variables are denoted by `{{ variable_name }}`

In [2]:
# Note, we could have this just a string, but as a dict allows us to add comments
def create_pipeline(
    input_path,
    output_path,
    polygon_wkt,
    manual_pre_norm_noise_expr: str | None = None,
    manual_post_norm_noise_expr: str | None = None,
) -> str:
    pipeline_template_dict = [
        # Read the input LAS file
        {"type": "readers.las", "filename": input_path},
        # Reproject to MGA2020 + Aus Height Datum
        {"type": "filters.reprojection", "out_srs": "EPSG:7855+5711"},
        # Crop to our site polygon
        # This is optional, but useful as our source sites are quite large
        # and we don't need the whole thing
        {"type": "filters.crop", "polygon": polygon_wkt},
        # Note, if you wanted to calculate your own ground classification points
        # do so here. We'll keep the ground classificaiton provided by VirtualTas.
        # e.g. { "type": "filters.csf", ... }
        # Calculate height above ground
        {"type": "filters.hag_nn"},
        # Label statistical outliers as noise (classification 7)
        {
            "type": "filters.outlier",
            "method": "statistical",
            "mean_k": 6,
            "multiplier": 10,
        },
        # Optionally, apply a manual noise filter expression
        # Save as a COPC file
        {
            "type": "writers.copc",
            "filename": output_path,
            "forward": "scale,offset",
            "extra_dims": "all",
        },
    ]

    if manual_pre_norm_noise_expr is not None:
        # Insert befor hag
        pipeline_template_dict.insert(
            3, {"type": "filters.assign", "value": manual_pre_norm_noise_expr}
        )

    if manual_post_norm_noise_expr is not None:
        # Insert the manual noise filter before the writer
        pipeline_template_dict.insert(
            -1, {"type": "filters.assign", "value": manual_post_norm_noise_expr}
        )

    return json.dumps(pipeline_template_dict, indent=2)

### Site pipelines

We use some data from our previously created `sites.geojson` to create the pdal pipelines.

In [3]:
import geopandas as gpd

sites_gdf = gpd.read_file("../data/outputs/sites/sites.geojson")
sites_gdf = sites_gdf.set_index('id')

sites_gdf.head()

,site,geometry
id,,
AGG_O_01,AGG_O_01,"POLYGON ((463303.79 5259716.755, 463123.329 52..."
AGG_O_05,AGG_O_05,"POLYGON ((455430.465 5284117.991, 455191.829 5..."
AGG_O_07,AGG_O_07,"POLYGON ((464747.381 5299156.759, 464706.037 5..."
AGG_Y_02,AGG_Y_02,"POLYGON ((491855.984 5230960.243, 491825.641 5..."
AGG_Y_03,AGG_Y_03,"POLYGON ((490748.752 5208804.286, 490664.016 5..."


There are 3 variables in the pipeline above:
- input_path - Where to source the lidar file for that site
- output_path - Where to save the processed lidar file
- polygon_wkt - The polygon for that site in well known text (WKT) format
- manual_noise_filter_expression - An optional PDAL expression to filter noise points for cloud noise found in ULM_325 and ULM_147

In [4]:
def center_and_size_to_box(center, size):
    (cx, cy, cz) = center
    (sx, sy, sz) = size
    return (cx - 0.5 * sx, cx + 0.5 * sx,
            cy - 0.5 * sy, cy + 0.5 * sy,
            cz - 0.5 * sz, cz + 0.5 * sz)

def get_ulm_325_expr():
    # The values of the box were manually determined by inspecting the point cloud
    # in cloudcompare.
    box_center = (476110.031, 5230827.372, 87.567)
    box_size = (53.144, 49.606, 36.957)

    clip_box = center_and_size_to_box(box_center, box_size)
    (minx, maxx, miny, maxy, minz, maxz) = clip_box

    post_assign_expressions = [
        f"Classification = 18 WHERE X >= {minx} && X <= {maxx} && Y >= {miny} && Y <= {maxy} && HeightAboveGround >= {minz} && HeightAboveGround <= {maxz}",
        "Classification = 18 WHERE HeightAboveGround >= 100"
    ]

    return (None, post_assign_expressions)

def get_ulm_147_expr():
    # Pre clips use Z

    pre_box_center = (457928.866, 5285531.132, 417.412)
    pre_box_size = (89.205, 143.393, 32.591)
    pre_clip_box = center_and_size_to_box(pre_box_center, pre_box_size)
    (minx, maxx, miny, maxy, minz, maxz) = pre_clip_box

    pre_assign_expressions = [
       f"Classification = 18 WHERE X >= {minx} && X <= {maxx} && Y >= {miny} && Y <= {maxy} && Z >= {minz} && Z <= {maxz}",
    ]

    pre_box_center = (457940.068, 5285580.835, 544.315)
    pre_box_size = (69.96, 39.172, 38.754)
    pre_clip_box = center_and_size_to_box(pre_box_center, pre_box_size)
    (minx, maxx, miny, maxy, minz, maxz) = pre_clip_box

    pre_assign_expressions.append(
        f"Classification = 18 WHERE X >= {minx} && X <= {maxx} && Y >= {miny} && Y <= {maxy} && Z >= {minz} && Z <= {maxz}",
    )

    # Post clips can use HeightAboveGround

    boxA_center = (457919.822, 5285528.094, 121.494)
    boxA_size = (95.724, 93.489, 82.994)

    boxB_center = (457945.263, 5285589.062, 120.843)
    boxB_size = (58.202, 22.717, 29.792)

    clip_boxA = center_and_size_to_box(boxA_center, boxA_size)
    (minxa, maxxa, minya, maxya, minza, maxza) = clip_boxA

    clip_boxB = center_and_size_to_box(boxB_center, boxB_size)
    (minxb, maxxb, minyb, maxyb, minzb, maxzb) = clip_boxB

    post_assign_expressions = [
        f"Classification = 18 WHERE X >= {minxa} && X <= {maxxa} && Y >= {minya} && Y <= {maxya} && HeightAboveGround >= {minza} && HeightAboveGround <= {maxza}",
        f"Classification = 18 WHERE X >= {minxb} && X <= {maxxb} && Y >= {minyb} && Y <= {maxyb} && HeightAboveGround >= {minzb} && HeightAboveGround <= {maxzb}",
        "Classification = 18 WHERE HeightAboveGround >= 100"
    ]

    return (pre_assign_expressions, post_assign_expressions)


In [5]:
data_dir = Path("../data")
lidar_source_dir = data_dir / "source" / "cycle-2"  # cycle-2 has best coverage
lidar_output_dir = data_dir / "outputs" / "sites" / "lidar"
lidar_output_dir.mkdir(parents=True, exist_ok=True)

def create_pipeline_from_site(site_row):
    site_id = site_row.name

    input_path = str(lidar_source_dir / f"{site_id}.laz")
    output_path = str(lidar_output_dir / f"{site_id}.copc.laz")
    polygon_wkt = site_row.geometry.wkt
    manual_pre_norm_noise_expr = None
    manual_post_norm_noise_expr = None

    if site_id == 'ULM_325':
        manual_pre_norm_noise_expr, manual_post_norm_noise_expr = get_ulm_325_expr()
    elif site_id == 'ULM_147':
        manual_pre_norm_noise_expr, manual_post_norm_noise_expr  = get_ulm_147_expr()

    return pd.Series({ "pipeline": create_pipeline(input_path, output_path, polygon_wkt, manual_pre_norm_noise_expr, manual_post_norm_noise_expr ) })

pipelines = sites_gdf.apply(create_pipeline_from_site, axis=1)

pipelines

,pipeline
id,
AGG_O_01,"[\n {\n ""type"": ""readers.las"",\n ""filen..."
AGG_O_05,"[\n {\n ""type"": ""readers.las"",\n ""filen..."
AGG_O_07,"[\n {\n ""type"": ""readers.las"",\n ""filen..."
AGG_Y_02,"[\n {\n ""type"": ""readers.las"",\n ""filen..."
AGG_Y_03,"[\n {\n ""type"": ""readers.las"",\n ""filen..."
...,...
ULO_271,"[\n {\n ""type"": ""readers.las"",\n ""filen..."
ULY_Y_231,"[\n {\n ""type"": ""readers.las"",\n ""filen..."
ULY_Y_232,"[\n {\n ""type"": ""readers.las"",\n ""filen..."


## Processing Pipelines

PDAL is built around processing these pipelines.

In [6]:
def process_pdal_pipeline(pipeline: str, return_data: bool = False):
    """
    Process a PDAL pipeline string.

    Args:
        pipeline (str): The PDAL pipeline JSON string.
        return_data (bool): If True, return the PDAL Pipeline object after execution. Defaults to False. Returning pipeline data
        will contain metadata and all the points processed by the pipeline. This can be a large object so defaults to False.
    """
    pipeline_obj = pdal.Pipeline(pipeline)
    count = pipeline_obj.execute()  # Execute the pipeline
    return (count, pipeline_obj if return_data else None)

Processing a single pipeline can take some time.

In [7]:
print(pipelines.loc['ULM_147'].pipeline)

[
  {
    "type": "readers.las",
    "filename": "../data/source/cycle-2/ULM_147.laz"
  },
  {
    "type": "filters.reprojection",
    "out_srs": "EPSG:7855+5711"
  },
  {
    "type": "filters.crop",
    "polygon": "POLYGON ((457757.96986439393 5285383.535780876, 457701.52150216635 5285453.632731068, 457748.5903740446 5285496.50854611, 457867.8860329714 5285593.24103093, 457873.11790809984 5285596.428013507, 457973.4029918211 5285603.12217709, 457979.397272664 5285513.3220177265, 457757.96986439393 5285383.535780876))"
  },
  {
    "type": "filters.assign",
    "value": [
      "Classification = 18 WHERE X >= 457884.2635 && X <= 457973.46849999996 && Y >= 5285459.435500001 && Y <= 5285602.8285 && Z >= 401.1165 && Z <= 433.7075",
      "Classification = 18 WHERE X >= 457905.08800000005 && X <= 457975.048 && Y >= 5285561.249 && Y <= 5285600.421 && Z >= 524.9380000000001 && Z <= 563.692"
    ]
  },
  {
    "type": "filters.hag_nn"
  },
  {
    "type": "filters.outlier",
    "method": "sta

In [8]:
%%time

p= pipelines.loc['ULM_147'].pipeline


(count, pl) = process_pdal_pipeline(p, return_data=True)
print(f"Processed {count} points.")

points = pl.arrays[0]
points_df = pd.DataFrame(pl.arrays[0])
points_df.head()

Processed 2076852 points.
CPU times: user 17.2 s, sys: 322 ms, total: 17.5 s
Wall time: 15.4 s


,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,Synthetic,...,ScanAngleRank,UserData,PointSourceId,GpsTime,ScanChannel,Red,Green,Blue,Infrared,HeightAboveGround
0,457958.591,5285501.252,462.596,28599,3,3,1,0,0,0,...,24.384001,104,2,4.128239e+08,0,8738,11565,16191,14005,19.798
1,457958.379,5285501.286,467.492,29697,3,3,1,0,0,0,...,24.480000,129,2,4.128239e+08,0,12079,13878,18504,14826,24.694
2,457957.978,5285501.355,469.988,28425,3,3,1,0,0,0,...,24.480000,141,2,4.128239e+08,0,10537,12336,16448,15509,27.190
3,457957.579,5285501.425,470.142,29117,3,3,1,0,0,0,...,24.480000,142,2,4.128239e+08,0,12336,14392,17990,15837,27.344
4,457958.365,5285501.283,474.054,29677,2,3,1,0,0,0,...,24.570000,161,2,4.128239e+08,0,11051,12850,16448,12589,31.256


### Parallel processing with Dask

In [9]:
from dask.distributed import Client

client = Client()  # Start a Dask client
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55216,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:55227,Total threads: 2
Dashboard: http://127.0.0.1:55229/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:55219,


In [10]:
%%time

futures = client.map(process_pdal_pipeline, pipelines['pipeline'].to_list(), key=pipelines.index.to_list())
results = client.gather(futures)

CPU times: user 42.8 s, sys: 9.02 s, total: 51.9 s
Wall time: 6min 38s


In [1]:
total_points = 0
for r in results:
    total_points += r[0]

f"Total points: {total_points:,}"

NameError: name 'results' is not defined